In [1]:
## Aim: For each site, mark 500 km buffer circle around it, and.. 
## ..count the number of fire event and total FRP within buffer_circle.

In [2]:
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
from functools import partial
import pyproj
from shapely.ops import transform
from shapely.geometry import Point

In [3]:
from shapely.geometry import Polygon

In [11]:
# Shape file of Fire data from FIRMS NASA
fp = r'C:/Users/LENOVO/Documents/Diwali_Impact_coding/Firecount/Country/MODIS_Country/fire_nrt_M-C61_332626.shp'
map_df = gpd.read_file(fp) # Fire dataframe

### Marking circle around a loaction (lat,lon) and its geometrical data.

In [12]:
proj_wgs84 = pyproj.Proj('+proj=longlat +datum=WGS84')

In [13]:
proj_wgs84

<Other Coordinate Operation Transformer: longlat>
Description: PROJ-based coordinate operation
Area of Use:
- undefined

In [14]:
## Function to create km circle around site(lat,lon).

def buffer_circle(lat, lon, km):
    
    aeqp = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0' # AEQP-Azimuthal equidistant projection
    project = partial(
        pyproj.transform,
        pyproj.Proj(aeqp.format(lat=lat, lon=lon)),
        proj_wgs84)
    buf = Point(0, 0).buffer(km * 1000)  # distance 
    return transform(project, buf).exterior.coords[:]

In [15]:
### Uploading locations (lat,lon) of sites.
Site_locations_data=pd.read_excel('DiwaliData_DataAllIndia_7D_Feb27.xlsx', sheet_name='Locations')

In [16]:
Lattitude=Site_locations_data['Lattitude']
Longitude=Site_locations_data['Longitude']

In [42]:
circle_pts=pd.DataFrame() # Dataframe to store lat,lon datapoints of circle marked.
result_data=pd.DataFrame([]) # Final output data.
for i in range(len(Site_locations_data)):
    
    #-----Using function to create 500 km circle around site(lat,lon).-----------------------
    circle_pts =pd.DataFrame(buffer_circle(Lattitude[i],Longitude[i],500))  
    circle_pts.rename(columns = {0:'Longitude',1:'Latitude'}, inplace = True)
   
    lon_point_list=circle_pts['Longitude'] # Lon of formed buffer_circle.
    lat_point_list=circle_pts['Latitude'] # Lat of formed buffer_circle.
    
    #---Converting circular_pts (lat,lon) data as geometry polygon data.------------------------
    polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
    polygon = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[polygon_geom])
    
    #---------Dataframe to store details of Fire data(map_df) which falls within polygon-----------
    new_df=pd.DataFrame([]) # Dataframe to store fire datapts. 
    for j in range(len(map_df)): # iterating over fire data to collect all points falling within polygon.
        Pt=Point(map_df['LONGITUDE'][j],map_df['LATITUDE'][j]) # Marking point from site location(lat,lon) detail.
        if Pt.within(polygon).bool()==True: # Checking whether falling in polygon, True-if falling.
            new_df=pd.concat([new_df,map_df[j:j+1]])
            
    # Extracting necessary/required details from new_df after done for particular site.---------------------
    temp=pd.DataFrame() # Dataframe to store temporarily details of every site.
    temp['FRP']=[new_df['FRP'].sum()] # Total FRP data
    temp['Count']=[new_df['FRP'].count()] # FRP count data
    temp['City']=Site_locations_data['City'][i] # City of site 
    
    result_data=pd.concat([result_data,temp])  # Concatenating details of every site, taking latest from 'temp'.

C:\Users\LENOVO\anaconda3\lib\site-packages\shapely\ops.py:278: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\LENOVO\anaconda3\lib\site-packages\shapely\ops.py:278: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\LENOVO\anaconda3\lib\site-packages\shapely\ops.py:278: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\LENOVO\anaconda3\lib\site-packages\shapely\ops.py:278: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-py

C:\Users\LENOVO\anaconda3\lib\site-packages\shapely\ops.py:278: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\LENOVO\anaconda3\lib\site-packages\shapely\ops.py:278: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\LENOVO\anaconda3\lib\site-packages\shapely\ops.py:278: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\LENOVO\anaconda3\lib\site-packages\shapely\ops.py:278: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-py

C:\Users\LENOVO\anaconda3\lib\site-packages\shapely\ops.py:278: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\LENOVO\anaconda3\lib\site-packages\shapely\ops.py:278: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\LENOVO\anaconda3\lib\site-packages\shapely\ops.py:278: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\LENOVO\anaconda3\lib\site-packages\shapely\ops.py:278: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-py

C:\Users\LENOVO\anaconda3\lib\site-packages\shapely\ops.py:278: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\LENOVO\anaconda3\lib\site-packages\shapely\ops.py:278: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\LENOVO\anaconda3\lib\site-packages\shapely\ops.py:278: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
C:\Users\LENOVO\anaconda3\lib\site-packages\shapely\ops.py:278: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-py

In [43]:
result_data # City, total FRP and fireevent Count happened within 500 km of sites.

,FRP,Count,City
0,1337.78,95,Amaravati
0,2214.17,160,Anantpur
0,1543.92,110,Tirupati
0,1319.49,94,Vijayawada
0,1458.62,122,Vishakhapatnam
...,...,...,...
0,3632.14,225,Ratlam
0,3689.39,226,Ujjain
0,2570.02,208,Bhilai
0,2801.27,232,Bilaspur
